In [19]:
import os
import sys
import pandas as pd
from importlib import reload

In [24]:
import enviserv.dictan
from enviserv.dictan import DictAnalyzer

In [4]:
import Connection as cn

In [5]:
spark = cn.spark_app_builder(cn.get_spark_master_ip(),
                             spark_app_name = "pyspark-taxi-forecasting")

INFO:ch:spark_master_ip: 172.18.0.2
DEBUG:ch:starting import SparkSession
INFO:ch:starting building spark app object: pyspark-taxi-forecasting
INFO:ch:builded spark app object: <pyspark.sql.session.SparkSession object at 0x7f748dbddc50>


In [6]:
%%time
taxi = spark.read.load('/work/data/Taxi_Trips_-_2022.csv', 
                       format='csv', header='true', inferSchema='true')

CPU times: user 12.7 ms, sys: 756 µs, total: 13.4 ms
Wall time: 43.1 s


In [7]:
%%time
# Вывести схему данных
taxi.printSchema()

root
 |-- Trip ID: string (nullable = true)
 |-- Taxi ID: string (nullable = true)
 |-- Trip Start Timestamp: string (nullable = true)
 |-- Trip End Timestamp: string (nullable = true)
 |-- Trip Seconds: integer (nullable = true)
 |-- Trip Miles: double (nullable = true)
 |-- Pickup Census Tract: long (nullable = true)
 |-- Dropoff Census Tract: long (nullable = true)
 |-- Pickup Community Area: integer (nullable = true)
 |-- Dropoff Community Area: integer (nullable = true)
 |-- Fare: double (nullable = true)
 |-- Tips: double (nullable = true)
 |-- Tolls: double (nullable = true)
 |-- Extras: double (nullable = true)
 |-- Trip Total: double (nullable = true)
 |-- Payment Type: string (nullable = true)
 |-- Company: string (nullable = true)
 |-- Pickup Centroid Latitude: double (nullable = true)
 |-- Pickup Centroid Longitude: double (nullable = true)
 |-- Pickup Centroid Location: string (nullable = true)
 |-- Dropoff Centroid Latitude: double (nullable = true)
 |-- Dropoff Centroid 

In [8]:
pd.set_option('display.max_colwidth', None)  # Показать полное содержимое ячеек столбцов

In [9]:
table_descr = pd.read_csv('/work/data/table_descr.csv',sep=';')

In [10]:
table_descr

,Column Name,Description,Type,Описание,Тип
0,Trip ID,A unique identifier for the trip.,Plain Text,Уникальный идентификатор поездки,Обычный текст
1,Taxi ID,A unique identifier for the taxi.,Plain Text,Уникальный идентификатор такси,Обычный текст
2,Trip Start Timestamp,When the trip started rounded to the nearest 15 minutes.,Date & Time,Время начала поездки округленное до ближайших 15 минут,Дата и время
3,Trip End Timestamp,When the trip ended rounded to the nearest 15 minutes.,Date & Time,Время окончания поездки округленное до ближайших 15 минут,Дата и время
4,Trip Seconds,Time of the trip in seconds.,Number,Продолжительность поездки в секундах,Число
5,Trip Miles,Distance of the trip in miles.,Number,Расстояние поездки в милях,Число
6,Pickup Census Tract,The Census Tract where the trip began. For privacy this Census Tract is not shown for some trips. This column often will be blank for locations outside Chicago.,Plain Text,Код района по переписи населения где началась поездка. В целях конфиденциальности этот код не отображается для некоторых поездок. Это поле часто остается пустым для мест за пределами Чикаго.,Обычный текст
7,Dropoff Census Tract,The Census Tract where the trip ended. For privacy this Census Tract is not shown for some trips. This column often will be blank for locations outside Chicago.,Plain Text,Код района по переписи населения где завершилась поездка. В целях конфиденциальности этот код не отображается для некоторых поездок. Это поле часто остается пустым для мест за пределами Чикаго.,Обычный текст
8,Pickup Community Area,The Community Area where the trip began. This column will be blank for locations outside Chicago.,Number,Номер коммунальной области где началась поездка. Это поле остается пустым для мест за пределами Чикаго.,Число
9,Dropoff Community Area,The Community Area where the trip ended. This column will be blank for locations outside Chicago.,Number,Номер коммунальной области где завершилась поездка. Это поле остается пустым для мест за пределами Чикаго.,Число


In [11]:
pd.reset_option('display.max_colwidth')  # Сброс максимальной ширины столбцов к значению по умолчанию

In [12]:
# %%time
# describe_result = taxi.describe().toPandas()

CPU times: user 447 ms, sys: 104 ms, total: 551 ms
Wall time: 2min 54s


In [15]:
# output_file_path = '/work/data/2022_describe.csv'
# describe_result.to_csv(output_file_path, index=False) #transpose().

In [12]:
describe_result_22 = pd.read_csv('/work/data/2022_describe.csv',sep=',')

In [32]:
def get_column_types(dataframe):
    col_types = {}  # Словарь для хранения типов данных каждого столбца
    for column in dataframe.columns:
        col_types[column] = str(dataframe[column].dtype)
        if col_types[column] == 'object':  # Если тип столбца 'object'
            column_values = dataframe[column]  # Получаем значения столбца
            column_val_types = {}  # Словарь для хранения типов значений в столбце
            for idx, val in column_values.items():
                if isinstance(val, int):  # Проверяем, является ли значение целым числом
                    column_val_types[idx] = 'int64'
                elif isinstance(val, float):  # Проверяем, является ли значение числом с плавающей точкой
                    column_val_types[idx] = 'float64'
                elif isinstance(val, str):  # Проверяем, является ли значение строкой
                    column_val_types[idx] = 'string'
                else:
                    column_val_types[idx] = type(val).__name__  # Возвращаем тип данных Python объекта
                # Переопределяем тип столбца на основе типов значений в ячейках
                col_types[column] = column_val_types
    return col_types

In [ ]:
df = describe_result_22
col_types = get_column_types(df)

In [72]:
# reload(enviserv.dictan)
# from enviserv.dictan import DictAnalyzer
# da = DictAnalyzer()
# da.print_dict(col_types)

In [59]:
def get_df_formated(df, round_num, round_str):
    for col in df.columns:
        if df[col].dtype == 'float64' or df[col].dtype == 'int64':
            df[col] = df[col].\
            apply(lambda x: ('{:,.%df}' % round_num).format(x).replace(',', "'") \
                  if pd.notnull(x) else x)
        else:
            for idx, val in df[col].items():
                if isinstance(val, str):
                    try:
                        val = float(val)
                        if val.is_integer():
                            val = int(val)
                        if isinstance(val, float):
                            df.loc[idx, col] = ('{:,.%df}' % round_num).format(val).replace(',', "'")
                        elif isinstance(val, int):
                            df.loc[idx, col] = ('{:,.%df}' % 0).format(val).replace(',', "'")
                    except ValueError:
                        df.loc[idx, col] = val[:round_str]
                    
                    

describe_result_22_formatted = describe_result_22.copy()
get_df_formated(describe_result_22_formatted, 2, 10)


In [74]:
pd.set_option('display.width', None)
pd.set_option('display.max_columns', None)

describe_result_22_formatted.transpose()

,0,1,2,3,4
summary,count,mean,stddev,min,max
Trip ID,6'382'425,NaN,NaN,000000bb18,ffffff1aae
Taxi ID,6'382'425,NaN,NaN,0041f8f0c9,fff84aa08a
Trip Start Timestamp,6'382'425,NaN,NaN,01/01/2022,12/31/2022
Trip End Timestamp,6'382'213,NaN,NaN,01/01/2022,12/31/2022
Trip Seconds,6'380'960.00,1'198.21,1'895.66,0.00,86'341.00
Trip Miles,6'382'369.00,6.19,8.00,0.00,2'967.54
Pickup Census Tract,2'623'831.00,17'031'468'160.38,368'945.90,17'031'010'100.00,17'031'980'100.00
Dropoff Census Tract,2'675'331.00,17'031'411'846.86,345'773.49,17'031'010'100.00,17'031'980'100.00
Pickup Community Area,5'868'572.00,32.35,25.20,1.00,77.00
